In [69]:

import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.6.0


In [70]:
import pandas as pd
data_path = pd.read_csv('/Users/federicaluciavinella/new.csv')

In [71]:
data_path
reviews = data_path.drop(data_path.columns[0], axis=1)

In [107]:
reviews.head()
print(len(reviews))
reviews.head()

9344


,mr,ref,label
0,"name[Cocum], eatType[coffee shop], food[Englis...",The Cocum is a cheap Chinese coffee shop that ...,1
1,"name[The Eagle], customer rating[5 out of 5], ...",The Eagle is in the city centre near to Café B...,0
2,"name[Cotto], eatType[coffee shop], food[Englis...",In the area of riverside near The Portland Arm...,0
3,"name[The Wrestlers], eatType[coffee shop], foo...",The Wrestlers is a family-friendly venue near ...,1
4,"name[The Punter], customer rating[average], ar...",The Punter is a family friendly place in the r...,1


In [88]:
# Inspecting some of the reviews
for i in range(5):
    print("mr #",i+1)
    print(reviews.mr[i])
    print("ref #",i+1)
    print(reviews.ref[i])
    print(reviews.label[i])
    print()

('mr #', 1)
name[Cocum], eatType[coffee shop], food[English], priceRange[£20-25], customer rating[high], familyFriendly[yes]
('ref #', 1)
The Cocum is a cheap Chinese coffee shop that is family friendly and has an average customer rating.
1
()
('mr #', 2)
name[The Eagle], customer rating[5 out of 5], area[city centre], familyFriendly[no], near[Café Brazil]
('ref #', 2)
The Eagle is in the city centre near to Café Brazil. Its not family-friendly and has a rating of 5 out of 5
0
()
('mr #', 3)
name[Cotto], eatType[coffee shop], food[English], priceRange[high], customer rating[1 out of 5], area[riverside], near[The Portland Arms]
('ref #', 3)
In the area of riverside near The Portland Arms is Cotto, it is a high priced coffee shop that has English food, and it has a 1-5 customer rating.
0
()
('mr #', 4)
name[The Wrestlers], eatType[coffee shop], food[English], priceRange[more than £30], area[riverside], familyFriendly[yes], near[Raja Indian Cuisine]
('ref #', 4)
The Wrestlers is a family-

In [74]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    

    # Format words and remove unwanted characters
    #text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [75]:
# Clean the summaries and texts
clean_summaries = []
for summary in reviews.mr:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("mr are complete.")

clean_texts = []
for text in reviews.ref:
    clean_texts.append(clean_text(text))
print("ref are complete.")

mr are complete.
ref are complete.


In [89]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(25):
    print("mr #",i+1)
    print(clean_summaries[i])
    print("ref #",i+1)
    print(clean_texts[i])
    print(reviews.label[i])
    print(' ')


('mr #', 1)
name cocum   eattype coffee shop   food english   pricerange £20 25   customer rating high   familyfriendly yes 
('ref #', 1)
cocum cheap chinese coffee shop family friendly average customer rating
1
 
('mr #', 2)
name the eagle   customer rating 5 out of 5   area city centre   familyfriendly no   near café brazil 
('ref #', 2)
eagle city centre near café brazil family friendly rating 5 5
0
 
('mr #', 3)
name cotto   eattype coffee shop   food english   pricerange high   customer rating 1 out of 5   area riverside   near the portland arms 
('ref #', 3)
area riverside near portland arms cotto high priced coffee shop english food 1 5 customer rating
0
 
('mr #', 4)
name the wrestlers   eattype coffee shop   food english   pricerange more than £30   area riverside   familyfriendly yes   near raja indian cuisine 
('ref #', 4)
wrestlers family friendly venue near sorrento low customer ratings
1
 
('mr #', 5)
name the punter   customer rating average   area city centre   familyfr

In [119]:
reviews_list = pd.DataFrame(
    {'mr': clean_summaries,
     'ref': clean_texts,
     'label': reviews.label
     
    })

In [120]:
reviews_list.head()

,label,mr,ref
0,1,name cocum eattype coffee shop food englis...,cocum cheap chinese coffee shop family friendl...
1,0,name the eagle customer rating 5 out of 5 ...,eagle city centre near café brazil family frie...
2,0,name cotto eattype coffee shop food englis...,area riverside near portland arms cotto high p...
3,1,name the wrestlers eattype coffee shop foo...,wrestlers family friendly venue near sorrento ...
4,1,name the punter customer rating average ar...,punter family friendly place riverside area hi...


In [121]:
reviews_list.to_csv('dataframe.csv')

In [122]:
data_path = pd.read_csv('/Users/federicaluciavinella/dataframe.csv')

In [123]:
data_path

,Unnamed: 0,label,mr,ref
0,0,1,name cocum eattype coffee shop food englis...,cocum cheap chinese coffee shop family friendl...
1,1,0,name the eagle customer rating 5 out of 5 ...,eagle city centre near café brazil family frie...
2,2,0,name cotto eattype coffee shop food englis...,area riverside near portland arms cotto high p...
3,3,1,name the wrestlers eattype coffee shop foo...,wrestlers family friendly venue near sorrento ...
4,4,1,name the punter customer rating average ar...,punter family friendly place riverside area hi...
5,5,0,name the phoenix customer rating 5 out of 5 ...,5 5 rated phoenix located riverside area
6,6,1,name the eagle eattype coffee shop food en...,eagle coffee shop eat chinese food cheaply fam...
7,7,1,name cocum eattype coffee shop food chines...,cocum family friendly coffee shop serves chine...
8,8,1,name the wrestlers eattype coffee shop foo...,wrestlers coffee shop located riverside near r...
9,9,0,name fitzbillies eattype coffee shop food ...,fitzbillies adult themed coffee shop located c...


In [130]:
#split data train/test

RANDOM_SEED =42
X_train, X_test = train_test_split(data_path, test_size=0.2, random_state=RANDOM_SEED)

X_train = X_train.drop(['label'], axis=1)
y_test = X_test['label']
X_test = X_test.drop(['label'], axis=1)
X_train = X_train.values
X_test = X_test.values
X_train.shape



(1869, 3)

In [ ]:
#build vocabulary of the sentences
